In [1]:
import os
import re
from os import listdir
from os.path import isdir, join
from PIL import Image
import numpy as np
from mtcnn import MTCNN

# Inicializa o detector MTCNN
detector = MTCNN()

In [2]:
def extrair_face(arquivo, tamanho=(160, 160)):
    """
    Extrai a face de uma imagem e redimensiona para o tamanho especificado.

    :param arquivo: Caminho do arquivo de imagem.
    :param tamanho: Tamanho final da face (largura, altura).
    :return: Imagem da face ou None se nenhuma face for detectada.
    """
    if not os.path.exists(arquivo):
        print(f"Arquivo não encontrado: {arquivo}")
        return None

    try:
        imagem = Image.open(arquivo).convert('RGB')
        pixels = np.asarray(imagem)
        resposta_faces = detector.detect_faces(pixels)

        if not resposta_faces:
            print(f"Nenhuma face detectada na imagem: {arquivo}")
            return None

        x1, y1, largura, altura = resposta_faces[0]['box']
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + largura, y1 + altura

        face_extraida = pixels[y1:y2, x1:x2]
        face_image = Image.fromarray(face_extraida).resize(tamanho)
        return face_image
    except Exception as e:
        print(f"Erro ao processar a imagem {arquivo}: {e}")
        return None

In [3]:
def extrair_numero(nome_arquivo):
    """
    Extrai números de um nome de arquivo usando regex.

    :param nome_arquivo: Nome do arquivo.
    :return: Número extraído ou 0 se nenhum número for encontrado.
    """
    match = re.search(r'\d+', nome_arquivo)
    return int(match.group()) if match else 0

In [4]:
def carregar_imagens(diretorio):
    """
    Carrega e ordena os caminhos completos das imagens de um diretório.

    :param diretorio: Caminho do diretório.
    :return: Lista de caminhos completos das imagens ordenados.
    """
    arquivos = [f for f in listdir(diretorio) if f.lower().endswith('.jpg')]
    arquivos_ordenados = sorted(arquivos, key=extrair_numero)
    return [join(diretorio, f) for f in arquivos_ordenados]

In [5]:
def carregar_fotos(diretorio_src, diretorio_target):
    """
    Processa as imagens de um diretório fonte e salva as faces no diretório alvo.

    :param diretorio_src: Diretório de origem.
    :param diretorio_target: Diretório de destino.
    """
    if not os.path.exists(diretorio_target):
        os.makedirs(diretorio_target)

    imagens_ordenadas = carregar_imagens(diretorio_src)
    for imagem in imagens_ordenadas:
        face = extrair_face(imagem)
        if face:
            nome_arquivo = os.path.basename(imagem)
            destino = join(diretorio_target, nome_arquivo)
            face.save(destino, "JPEG", quality=100, optimize=True, progressive=True)

In [6]:
def carregar_diretorio(diretorio_src, diretorio_target):
    """
    Processa os subdiretórios de um diretório fonte e organiza as faces no destino.

    :param diretorio_src: Diretório de origem.
    :param diretorio_target: Diretório de destino.
    """
    for subdir in listdir(diretorio_src):
        path_src = join(diretorio_src, subdir)
        path_tgt = join(diretorio_target, subdir)

        if not isdir(path_src):
            continue

        if not os.path.exists(path_tgt):
            os.makedirs(path_tgt)

        carregar_fotos(path_src, path_tgt)

In [7]:
if __name__ == '__main__':
    # Diretórios de origem e destino    F:\Bootcamp_DIO\desafio_reconhecer_rostos\desafio_reconhecer_rostos
    diretorio_src = 'F:\\Bootcamp_DIO\\desafio_reconhecer_rostos\\img_entrada\\'
    diretorio_target = 'F:\\Bootcamp_DIO\\desafio_reconhecer_rostos\\img_saida\\'

    carregar_diretorio(diretorio_src, diretorio_target)

1/1 [==============================] - 0s 50ms/step
Nenhuma face detectada na imagem: F:\Bootcamp_DIO\desafio_reconhecer_rostos\img_entrada\desconhecido\desconhecido112.jpg
1/1 [==============================] - 0s 85ms/step
